# GQT-324

Behave allure report 會計算 untested 的數量，但 untested 並不會影響 'final result' (Jenkins 亮綠燈/黃燈)。 <br>
allure report 的 test case status (<font color=green>passd</font>/<font color=red>failed</font>/<font color=orange>broken</font>...) 是以每個 scenario 內是否有 step passed/failed/broken 判斷 (並非是從 scenario 的 status)。 <br>
所以如果在執行 step 之前發生 Error，便不會執行 step，此時 step 的 status 會保持預設值(untested)，也就說並不算是 Error。 <br>

**導致發生 Error 時，結果為 'untested'，final result 為 passed 的情況**

E.g., 錯誤發生在 before_scenario 的 hook 上
```shell
$ HOOK-ERROR in before_scenario: Exception: ...
$ ...
$ 0 features passed, 1 failed, 0 skipped
$ 0 scenarios passed, 1 failed, 0 skipped
$ 0 steps passed, 0 failed, 0 skipped, 0 undefined, 3 untested
```
allure report 會顯示 3 <font color=gray>untested</font>，final result 為 <font color=green>Passed</font>


## Resolution
### ~~1. 修改 test result 讓 allure 把 untested -> undefined~~ <font color=red>此方法無法解決 發生在 feature 的 hook-error</font>
在 after 的 hook (after_feature, after_scenario) 加上 hook_failed 的判斷 (feature/scenario 皆有存 hook_failed)， <br>
將第一個 step 設為 undefined (allure 會顯示 <font color=orange>broken</font>) <br>
e.g.
```python
if scenario.hook_failed == True:
    # scenario.all_steps 為 list_iterator，如果 scenario 中沒有任何 step，則為 None，
    # 所以不用 next() 取第一個 step，否則會跳 Exception 
    for step in scenario.all_steps:    
        # 把第一個 step 設為 undefined 即可
        if step.status == Status.untested:
            step.status = Status.undefined
            break
```

### 2. 透過 batch_trigger 檢查 html report
html report 不同於 allure report，我們可以直接讀取 final result 的資訊。 <br>
我們可以在 batch_trigger.py 內加上一段檢查，如果發現有 unteseted 的 step，則回傳 False 給 Jenkins <br>
Jenkins 檢查到錯誤後，將 job 的 status 改成 'Unstable'，即可避免出現錯誤而 job status 仍為 'Passed'



有關 behave result 和 allure report 的 mapping
Behave    |Allure
:--------:|:--------:
passed    |passed
failed    |failed
skipped   |skipped
undefined |broken
untested  |skipped